## Задание 5
### Обучение с подкреплением

Требуется реализовать метод обучения Q-learning или Deep Q-learning для любой из сред, реализованных в OpenAI Gym (кроме Discrete Lunar Lander). 
Результатом задания является записная книжка Jupyter Notebook в которой: 
 1. Создается выбранная среда OpenAI Gym 
 2. Агент обучается методом Q-learning 
 3. Визуализируется (средствами OpenAI Gym) поведение обученного агента 

In [1]:
import gym

env = gym.make("MountainCar-v0") # инициализируем среду с игрой MountainCar
env.reset()

import time
env.reset()
for i in range(1000):
    env.render()
    a = env.action_space.sample()
    new_state, reward, done, _ = env.step(a) # take a random action
    print(reward)
    if done:
        break
    time.sleep(0.1)
env.close()

-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0


In [2]:
NA = 3

In [3]:
NS = 2

In [4]:
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.activations import relu, linear

model = Sequential()
model.add(Dense(64, input_dim=NS, activation=relu))
model.add(Dense(64, activation=relu))
model.add(Dense(NA, activation=linear))
model.compile(loss='mse', optimizer=Adam(0.001))

In [5]:
import numpy as np

epsilon = 1.0
epsilon_decay = 0.998

def choose_action(state):
    if np.random.rand() <= epsilon:
        return random.randrange(NA)
    qs = model.predict(state)
    return np.argmax(qs[0])

In [6]:
import random
from collections import deque

memory = deque() # tuples (state, action, reward, next state, done)
batch_size = 64

gamma = 0.99 # discount factor

def update_model():
    global epsilon, epsilon_decay 
    if len(memory) < batch_size:
        return
    
    batch = random.sample(memory, batch_size)
    states = np.squeeze(np.array([i[0] for i in batch]))
    actions = np.array([i[1] for i in batch])
    rewards = np.array([i[2] for i in batch])
    next_states = np.squeeze(np.array([i[3] for i in batch]))
    dones = np.array([i[4] for i in batch])
    
    targets = model.predict_on_batch(states)
    targets[[i for i in range(len(actions))], [actions]] = rewards + gamma*(np.amax(model.predict_on_batch(next_states), axis=1))*(1-dones)
    
    model.fit(states, targets, epochs=1, verbose=0)
    if epsilon > 0.01:
        epsilon *= epsilon_decay

In [7]:
scores = []
for e in range(400):
    state = np.reshape(env.reset(), (1,2))
    score = 0
    for i in range(3000):
        action = choose_action(state)
        env.render()
        next_state, reward, done, _ = env.step(action)
        score += reward
        next_state = np.reshape(next_state, (1, 2))
        memory.append((state, action, reward, next_state, done))
        state = next_state
        update_model()
        if done:
            print("episode {} step {} => score {}".format(e, i, score))
            break
        scores.append(score)

        if np.mean(scores[-100:]) > 200:
            prinf("converged!")
            break

episode 0 step 199 => score -200.0
episode 1 step 199 => score -200.0
episode 2 step 199 => score -200.0
episode 3 step 199 => score -200.0
episode 4 step 199 => score -200.0
episode 5 step 199 => score -200.0
episode 6 step 199 => score -200.0
episode 7 step 199 => score -200.0
episode 8 step 199 => score -200.0
episode 9 step 199 => score -200.0
episode 10 step 199 => score -200.0
episode 11 step 199 => score -200.0
episode 12 step 199 => score -200.0
episode 13 step 199 => score -200.0
episode 14 step 199 => score -200.0
episode 15 step 199 => score -200.0
episode 16 step 199 => score -200.0
episode 17 step 199 => score -200.0
episode 18 step 199 => score -200.0
episode 19 step 199 => score -200.0
episode 20 step 199 => score -200.0
episode 21 step 199 => score -200.0
episode 22 step 199 => score -200.0
episode 23 step 199 => score -200.0
episode 24 step 199 => score -200.0
episode 25 step 199 => score -200.0
episode 26 step 199 => score -200.0
episode 27 step 199 => score -200.0
ep